### This code parse crime data for all 50 states from disaster center crime (https://www.disastercenter.com/crime/). The process includes parsing, raw data cleaning, effective data extracting, data cleaning (remove syntax, filter out Nans, data type convert), and merge to master sheet 

### The Key for the data set is State (In upper 2 letters state code, e.g. 'AL', 'CA')

### Powered by requestsm BeautifulSoup for parsing

In [1]:
import requests
from bs4 import BeautifulSoup
import string
import pandas as pd
import re

In [2]:
def fetch_and_parse_content(url):
    # fetch content from url
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print("Error: Unable to fetch content")
        return None

def parse_html_to_dataframe(html_content): # used for 1960 - 2009
    # Parse HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('tr')

    # Extract data from rows
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [clean_value(col.text.strip()) for col in cols]
        #prin(cols)
        data.append(cols)

    # Remove any empty rows
    data = [row for row in data if len(row) > 0]
    #print(data)

    # Column names
    column_names = ['Year', 'Population', 'Index', 'Violent', 'Property', 'Murder', 'Rape', 'Robbery', 'Aggravated Assault', 'Burglary', 'Larceny-Theft', 'Vehicle Theft']

    # Create DataFrame
    try:
        df = pd.DataFrame(data, columns=column_names)
    except:
        print(data)

    # Remove the first row, which contains the column names
    df = df.iloc[1:]

    # Drop rows with missing values
    df = df.dropna()

    return df

def clean_value(value):
    return re.sub(r'\s*,\s*', ',', value)

def parse_html_to_dataframe_v2(html_content): # this is used for 2010 -2019 because of the different format
    # Parse HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('tr')

    # Extract data from rows
    """
    data = []
    for row in rows:
        cols = row.find_all(['td', 'small'])
        cols = [clean_value(col.text.strip()) for col in cols]
        # Remove duplicates in each row
        cols = cols[::2]
        data.append(cols)

    # Remove any empty rows
    data = [row for row in data if len(row) > 0]

    # Column names
    column_names = ['Year', 'Population', 'Index', 'Violent', 'Property', 'Murder', 'Rape', 'Robbery', 'Aggravated Assault', 'Burglary', 'Larceny-Theft', 'Vehicle Theft']

    # Create DataFrame
    try:
        df = pd.DataFrame(data, columns=column_names)
    except:
        for d in data:
            print(len(d))
        print(data)
    """

    data = []
    for row in rows:
        cols = row.find_all(['td'])
        cleaned_cols = []
        for col in cols:
            innermost = col
            while innermost.find(['small', 'big']):
                innermost = innermost.find(['small', 'big'])
            cleaned_cols.append(clean_value(innermost.text.strip()))
        data.append(cleaned_cols)

    df = pd.DataFrame(data)

    # Remove the first row, which contains the column names
    df = df.iloc[1:]

    # Drop rows with missing values
    df = df.dropna()

    return df

def extract_effect_parts(html_content):
    all_content = str(html_content)
    effect_parts = all_content.split("""</tr>
              </tbody>
            </table>
            </center>
            </td>
          </tr>
          <tr>
            <td style="text-align: center;">
            <table style="text-align: left; width: 100%;" border="0"
 cellpadding="0" cellspacing="0">
              <tbody>
                <tr>""")[0]
    #print(effect_parts)
    try:
        effect_parts = effect_parts.split("""Number of Crimes""")[1]
    except:
        try: 
            effect_parts = effect_parts.split("""Population and Number""")[1]
        except:
            try:
                effect_parts = effect_parts.split("""and Number of""")[1]
            except:
                try: 
                    effect_parts = effect_parts.split("""Number of""")[1]
                except:
                    print("Error: Unable to parse content")
    effect_parts = effect_parts.split("""Forcible""")
    return effect_parts

def create_crime_dataframe(effect_parts):
    crime_master = pd.DataFrame(columns=['Year', 'Population', 'Index', 'Violent', 'Property', 'Murder', 'Rape', 'Robbery', 'Aggravated Assault', 'Burglary', 'Larceny-Theft', 'Vehicle Theft'])
    
    for i in range(1, 6):
        temp_df = parse_html_to_dataframe(effect_parts[i])
        # Add temp_df to master sheet
        crime_master = pd.concat([crime_master, temp_df])

    # Process 2010-2019 because of different format
    temp_df = parse_html_to_dataframe_v2(effect_parts[6].split('</tbody>')[0])
    temp_df.columns = ['Year', 'Population', 'Index', 'Violent', 'Property', 'Murder', 'Rape', 'Robbery', 'Aggravated Assault', 'Burglary', 'Larceny-Theft', 'Vehicle Theft']
    crime_master = pd.concat([crime_master,temp_df])
   
    return crime_master

def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))


### Test

In [ ]:
vars = pd.DataFrame(columns=['Year', 'State', 'Population', 'Index', 'Violent', 'Property', 'Murder', 'Rape', 'Robbery', 'Aggravated Assault', 'Burglary', 'Larceny-Theft', 'Vehicle Theft'])
nj_url = "https://www.disastercenter.com/crime/njcrimn.htm"
nj_content = fetch_and_parse_content(nj_url)
nj_effect_content = extract_effect_parts(nj_content)
nj_df = create_crime_dataframe(nj_effect_content)
nj_df["State"] = "NJ"

In [ ]:
nj_df

,Year,Population,Index,Violent,Property,Murder,Rape,Robbery,Aggravated Assault,Burglary,Larceny-Theft,Vehicle Theft,State
1,1960,"6,066,782","90,441","6,932","83,509",164,442,"2,591","3,735","25,698","45,903","11,908",NJ
2,1961,"6,244,000","96,057","6,682","89,375",153,458,"2,371","3,700","29,329","47,145","12,901",NJ
3,1962,"6,245,000","108,899","7,874","101,025",187,510,"2,816","4,361","31,868","54,516","14,641",NJ
4,1963,"6,470,000","122,867","8,849","114,018",181,561,"3,367","4,740","35,993","61,199","16,826",NJ
5,1964,"6,682,000","137,594","10,456","127,138",207,609,"3,812","5,828","40,143","68,072","18,923",NJ
6,1965,"6,774,000","138,636","10,422","128,214",219,605,"3,753","5,845","42,113","66,177","19,924",NJ
7,1966,"6,898,000","160,622","11,168","149,454",240,640,"4,397","5,891","49,176","77,374","22,904",NJ
8,1967,"7,003,000","191,720","13,204","178,516",276,677,"5,777","6,474","60,321","88,412","29,783",NJ
9,1968,"7,078,000","230,341","16,711","213,630",358,801,"8,747","6,805","71,621","105,309","36,700",NJ
10,1969,"7,148,000","230,927","17,226","213,701",369,914,"9,657","6,286","68,123","108,336","37,242",NJ


In [ ]:
ny_url = "https://www.disastercenter.com/crime/nycrime.htm"
ny_content = fetch_and_parse_content(ny_url)
ny_effect_content = extract_effect_parts(ny_content)
ny_df = create_crime_dataframe(ny_effect_content)
ny_df["State"] = "NY"

In [ ]:
ny_df

,Year,Population,Index,Violent,Property,Murder,Rape,Robbery,Aggravated Assault,Burglary,Larceny-Theft,Vehicle Theft,State
1,1965,"18,073,000","554,050","58,802","495,248",836,"2,320","28,182","27,464","183,443","253,353","58,452",NY
2,1966,"18,258,000","609,465","62,561","546,904",882,"2,439","30,098","29,142","196,127","286,409","64,368",NY
3,1967,"18,336,000","692,528","75,124","617,404",996,"2,665","40,202","31,261","219,157","314,472","83,775",NY
4,1968,"18,113,000","829,453","98,515","730,938","1,185","2,527","59,857","34,946","250,918","375,143","104,877",NY
5,1969,"18,321,000","837,210","105,870","731,340","1,324","2,902","64,754","36,890","248,477","367,463","115,400",NY
1,1970,"18,190,740","904,314","124,613","779,701","1,444","2,875","81,149","39,145","267,474","386,553","125,674",NY
2,1971,"18,391,000","935,022","145,048","789,974","1,823","3,225","97,682","42,318","273,704","388,612","127,658",NY
3,1972,"18,366,000","804,605","138,542","666,063","2,026","4,199","86,391","45,926","239,886","321,096","105,081",NY
4,1973,"18,265,000","814,349","135,468","678,881","2,040","4,852","80,795","47,781","246,246","320,307","112,328",NY
5,1974,"18,111,000","911,703","145,427","766,276","1,919","5,240","86,814","51,454","271,824","390,357","104,095",NY


### Define All State Lowercase Abbreviations For Scraping All Data

In [3]:

state_abbs = [
    'al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'fl', 'ga',
    'hi', 'id', 'il', 'in', 'ia', 'kn', 'ky', 'la', 'me', 'md',
    'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj',
    'nm', 'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc',
    'sd', 'tn', 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy'
]



### Main part

In [8]:
master = pd.DataFrame(columns=['Year', 'State', 'Population', 'Index', 'Violent', 'Property', 'Murder', 'Rape', 'Robbery', 'Aggravated Assault', 'Burglary', 'Larceny-Theft', 'Vehicle Theft'])
for s in state_abbs:
    print(s)
    if s in ["ms","mo","mt","ne","nj","nm","nc","nd","ok"]:
        url = "https://www.disastercenter.com/crime/{state}crimn.htm".format(state = s)
    else:
        url = "https://www.disastercenter.com/crime/{state}crime.htm".format(state = s)
    content = fetch_and_parse_content(url)
    effect_content = extract_effect_parts(content)
    to_add = create_crime_dataframe(effect_content)
    
    to_add["State"] = s.upper()
    #print(to_add.columns)
    master = pd.concat([master,to_add],ignore_index=True)

for col in master.columns:
    if col == "State":
        continue
    master[col] = master[col].apply(remove_punctuation).astype(int)
master.sample(30)

al
ak
az
ar
ca
co
ct
de
fl
ga
hi
id
il
in
ia
kn
ky
la
me
md
ma
mi
mn
ms
mo
mt
ne
nv
nh
nj
nm
ny
nc
nd
oh
ok
or
pa
ri
sc
sd
tn
tx
ut
vt
va
wa
wv
wi
wy


,Year,State,Population,Index,Violent,Property,Murder,Rape,Robbery,Aggravated Assault,Burglary,Larceny-Theft,Vehicle Theft
267,1986,CA,26981000,1824669,248370,1576299,3038,12119,92512,140701,457698,913004,205597
323,1982,CO,3045000,215584,15354,200230,182,1356,4587,9229,53260,134873,12097
908,1967,KN,2275000,56295,3248,53047,90,243,959,1956,13610,35231,4206
2511,2015,TN,6595056,236863,40816,196047,418,2032,7499,30098,43753,139259,13035
404,2003,CT,3486960,103026,11045,92981,112,698,4212,6023,16043,65043,11370
1,1961,AL,3302000,38105,5564,32541,427,252,630,4255,11205,18801,2535
2286,1970,RI,949723,40542,1944,38598,30,34,744,1136,9677,20763,8158
2193,1997,OR,3243000,203328,14412,188916,95,1306,3811,9200,33507,136129,19280
372,1971,CT,3081000,112338,5968,106370,96,367,2563,2942,32884,56875,16611
2505,2009,TN,6296254,279090,41933,237157,468,2019,9653,29793,63832,158309,15016


In [170]:
master.to_csv("./crime_master.csv")